In [1]:
import math

import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

import matplotlib.pyplot as plt
%matplotlib inline

from cipherbreak import *

c8a = open('8a.ciphertext').read()
c8b = open('8b.ciphertext').read()

c8as = sanitise(c8a)
c8bs = sanitise(c8b)


In [2]:
wap = open('../war-and-peace.txt').read()
len(wap)

3226615

In [3]:
c8bc = [ch for ch in cat(c for c in c8b if c in '012').split('2') if ch]
c8bc[:10]

['10', '00', '00', '01', '00', '111', '000', '100', '010', '101']

In [4]:
c8bchunks = chunks(c8bc, 5, fillvalue=[[]])
c8bchunks[:10]

[['10', '00', '00', '01', '00'],
 ['111', '000', '100', '010', '101'],
 ['100', '110', '011', '011', '000'],
 ['0010111', '1010011', '1100011', '1100000', '0000001'],
 ['0111', '0001', '0001', '0101', '1100'],
 ['0101', '0001', '0011', '0110', '1100'],
 ['1', '0', '0', '0', '0'],
 ['011', '110', '101', '000', '001'],
 ['00', '00', '01', '11', '10'],
 ['0010', '0010', '0100', '0100', '0000']]

In [5]:
c8bchars = [chunks(column_transposition_encipher(cat(char), 'abcde', fillcolumnwise=True, emptycolumnwise=False), 5)
 for char in c8bchunks]
c8bchars[:10]

[['10000', '00010'],
 ['10101', '10010', '10001'],
 ['11000', '01110', '00110'],
 ['01110', '00110', '11000', '00000', '10000', '11100', '11101'],
 ['00001', '10011', '10000', '11110'],
 ['00001', '10011', '00110', '11100'],
 ['10000'],
 ['01100', '11000', '10101'],
 ['00011', '00110'],
 ['00000', '00110', '11000', '00000']]

In [6]:
c8pos = [[int(cat(reversed(c)), 2) for c in bword] for bword in c8bchars]
max(n for ns in c8pos for n in ns), min(n for ns in c8pos for n in ns)

(25, 0)

In [7]:
c8text = wcat(cat(unpos(int(cat(reversed(c)), 2)) for c in bword) for bword in c8bchars)
c8text[:100]

'bi vjr dom omdabhx qzbp qzmh b gdv ym amda dxdbh ebffma qtbnm hjt yv qzm kjpq abxbqdf prosmbffdhnm p'

In [8]:
len(sanitise(c8text))

5325

In [9]:
eng_target = sanitise(wap)[:5325]
len(eng_target)

5325

In [10]:
random_target = cat(random.choice(string.ascii_lowercase) for _ in range(5325))
len(random_target)

5325

In [11]:
Ptrigrams(sanitise(c8text)), Ptrigrams(sanitise(eng_target)), Ptrigrams(sanitise(random_target))

(-32687.168128732188, -20426.14140719279, -30376.82045021536)

In [12]:
(21000 - 32700) / math.log(0.75)

40669.896112351824

In [13]:
(21000 - 22000) / math.log(0.75)

3476.059496782207

In [14]:
(21000 - 32700) / math.log(0.0075)

2391.2432630309268

In [15]:
math.exp((21000 - 327) / 400)

2.788846178301471e+22

In [16]:
math.exp(((-32000) - (-30000)) / 40000)

0.951229424500714

In [17]:
(-500) / math.log(0.75)

1738.0297483911036

In [18]:
ltrs = [p[0] for p in english_counts.most_common()]
cat(ltrs)

'etoainhsrdlumwycfgpbvkxjqz'

In [19]:
ctls =  [p[0] for p in collections.Counter(sanitise(c8text)).most_common()]
cat(ctls)

'mqdbjhopzafrntgxvkiyseuclw'

In [20]:
trans = {pr[0]: pr[1] for pr in zip(ctls, ltrs)}
tt = ''.maketrans(trans)
print(c8text.translate(tt))

ap fiu ohe heodanc tras tren a yof be deod ocoan kalled twame niw bf tre gist dacatol suhveallonme sfndamote ap a oy stall olave tren fiu yof be yf inlf rige ap tref rove kalled ye tren tras letteh wall rove been dasthabuted an unenmhfgted pihy ti oll tre simaol yedao sates ond oll tre news ocenmaes an tre wihld treah noye wall be kniwn ti evehfine ond tre stihf ip wrot tref thaed ti di wall be reodlane news ap nit tren yf rige as trot fiu wall keeg tras quaet pih niw pih fiuh soke os well os pih yane yf noye as joyelao dolyoh ond a oy gohtlf hesginsable pih tras yess tras letteh as yf otteygt ti exgloan yf hile an wrot roggened an tre rige trot nitranc lake at wall eveh roggen ocoan tre sfndamote wos o civehnyent anataotave ti ghegohe pih o wihld an wramr enmhfgtain wall be si semuhe trot ni ine mon mhomk at trot wihld as olyist rehe ond tre quontuy temrnilicf hevilutain as lakelf ti cave bahtr ti at siin ap nitranc mronces ghavomf wall be obsilute ond si wall semhemf riw fiu peel obi

In [21]:
Ptrigrams(sanitise(c8text.translate(tt)))

-21381.986189863124

In [22]:
c8alphabet0 = cat(trans[l] for l in sorted(trans))
c8alphabet0

'dajoklynpigqemhstuvwxfzcbr'

In [23]:
# logger.setLevel(logging.DEBUG)
logger.setLevel(logging.WARN)

In [30]:
cipher_alphabet, score = simulated_annealing_break(
    sanitise(c8text), workers=3,
    initial_temperature=200, max_iterations=20000, 
    plain_alphabet=cat(ltrs), cipher_alphabet=cat(ctls),
    fitness=Ptrigrams)
cipher_alphabet, score

('mqjdbhzpoafrgtvnixkyseuclw', -17922.293053151883)

In [31]:
cipher_translation = ''.maketrans(cipher_alphabet, cat(ltrs))
c8text.translate(cipher_translation)

'if you are reading this then i may be dead again killed twice now by the post digital surveillance syndicate if i am still alive then you may be my only hope if they have killed me then this letter will have been distributed in unencrypted form to all the social media sites and all the news agencies in the world their name will be known to everyone and the story of what they tried to do will be headline news if not then my hope is that you will keep this quiet for now for your sake as well as for mine my name is jamelia dalmar and i am partly responsible for this mess this letter is my attempt to explain my role in what happened in the hope that nothing like it will ever happen again the syndicate was a government initiative to prepare for a world in which encryption will be so secure that no one can crack it that world is almost here and the quantum technology revolution is likely to give birth to it soon if nothing changes privacy will be absolute and so will secrecy how you feel ab

In [26]:
# cipher_alphabet, score = simulated_annealing_break_worker(
#     sanitise(c8text), 
#     ltrs, ctls,
#     700, 
#     200,
#     Ptrigrams)
# cipher_alphabet, score

In [27]:
{chr(v): chr(k) for k, v in cipher_translation.items()}

{'a': 'd',
 'b': 'y',
 'c': 'n',
 'd': 'a',
 'e': 'm',
 'f': 'i',
 'g': 'x',
 'h': 'z',
 'i': 'b',
 'j': 'c',
 'k': 'e',
 'l': 'f',
 'm': 'g',
 'n': 'h',
 'o': 'j',
 'p': 'k',
 'q': 'l',
 'r': 'o',
 's': 'p',
 't': 'q',
 'u': 'r',
 'v': 's',
 'w': 't',
 'x': 'u',
 'y': 'v',
 'z': 'w'}

In [28]:
keyword_decipher(c8text, 'dynamix', KeywordWrapAlphabet.from_last)

'if you are reading this then i may be dead again killed twice now by the post digital surveillance syndicate if i am still alive then you may be my only hope if they have killed me then this letter will have been distributed in unencrypted form to all the social media sites and all the news agencies in the world their name will be known to everyone and the story of what they tried to do will be headline news if not then my hope is that you will keep this quiet for now for your sake as well as for mine my name is jamelia dalmar and i am partly responsible for this mess this letter is my attempt to explain my role in what happened in the hope that nothing like it will ever happen again the syndicate was a government initiative to prepare for a world in which encryption will be so secure that no one can crack it that world is almost here and the quantum technology revolution is likely to give birth to it soon if nothing changes privacy will be absolute and so will secrecy how you feel ab

In [29]:
cipher_alphabet, score = monoalphabetic_break_hillclimbing_mp(
    sanitise(c8text), workers=3,
    max_iterations=20000, 
    plain_alphabet=cat(ltrs), cipher_alphabet=cat(ctls),
    fitness=Ptrigrams)
cipher_alphabet, score

('mqjdbhzpoafrgtvnixkyseuclw', -17922.293053151883)

In [32]:
cipher_alphabet, score = simulated_annealing_break(
    sanitise(c8text), workers=3,
    initial_temperature=200, max_iterations=20000, 
#     plain_alphabet=cat(ltrs), cipher_alphabet=cat(ctls),
    fitness=Ptrigrams)
cipher_alphabet, score

('dynamixzbcefghjklopqrstuvw', -17922.293053151883)

In [34]:
cipher_translation = ''.maketrans(cipher_alphabet, string.ascii_lowercase)
c8text.translate(cipher_translation)

'if you are reading this then i may be dead again killed twice now by the post digital surveillance syndicate if i am still alive then you may be my only hope if they have killed me then this letter will have been distributed in unencrypted form to all the social media sites and all the news agencies in the world their name will be known to everyone and the story of what they tried to do will be headline news if not then my hope is that you will keep this quiet for now for your sake as well as for mine my name is jamelia dalmar and i am partly responsible for this mess this letter is my attempt to explain my role in what happened in the hope that nothing like it will ever happen again the syndicate was a government initiative to prepare for a world in which encryption will be so secure that no one can crack it that world is almost here and the quantum technology revolution is likely to give birth to it soon if nothing changes privacy will be absolute and so will secrecy how you feel ab